In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 📘 Preliminaries: The Mathematics of Economic Growth

Understanding economic growth requires familiarity with a few key mathematical concepts, particularly how to measure and manipulate growth rates.

We often want to know:
1.  How much did a variable (like GDP per capita) change between two points in time?
2.  What was the average annual rate of change over a longer period?
3.  How can we simplify calculations involving growth rates?

### 📈 Percent Changes and Growth Rates

- **Percent Change:**  
  The fundamental measure of change relative to an initial value:
  
  $$
  \text{Percent Change} = \frac{\text{New Value} - \text{Old Value}}{\text{Old Value}} \times 100\%
  $$
  
  or equivalently:
  
  $$
  \text{Percent Change} = \left( \frac{Y_{t+1} - Y_t}{Y_t} \right) \times 100\%
  $$

- **Growth Rate (g):**  
  This usually refers to the percent change expressed as a decimal (i.e., divided by 100). It represents the factor by which a variable changes per period:
  
  $$
  g = \frac{Y_{t+1} - Y_t}{Y_t}
  $$
  
  This implies:
  
  $$
  Y_{t+1} = Y_t + g Y_t = Y_t (1 + g)
  $$

- **Calculating Level after Growth:**  
  If a variable $begin:math:text$ Y_0 $end:math:text$ grows at a constant rate $begin:math:text$ g $end:math:text$ for $begin:math:text$ T $end:math:text$ periods, its value at time $begin:math:text$ T $end:math:text$ is:
  
  $$
  Y_T = Y_0 (1 + g)^T
  $$

# 🗓️ Annualized Growth Rates

Often, we know the value of a variable at two points in time far apart (e.g., GDP in 1980 and 2020) and want to find the *average annual growth rate* ($g$) that would connect them.

We rearrange the formula $Y_T = Y_0 (1 + g)^T$:
$$ \frac{Y_T}{Y_0} = (1 + g)^T $$
$$ \left( \frac{Y_T}{Y_0} \right)^{1/T} = 1 + g $$
$$ g_{\text{annualized}} = \left( \frac{Y_T}{Y_0} \right)^{1/T} - 1 $$
Where $T$ is the number of years between the measurement of $Y_0$ and $Y_T$.

# 🪵 Using Logarithms (A Useful Approximation)

Natural logarithms (ln) are extremely useful in growth economics because the change in the log of a variable is approximately equal to its percentage change (or growth rate) when the change is small:

$$ g = \frac{Y_{t+1} - Y_t}{Y_t} \approx \ln(Y_{t+1}) - \ln(Y_t) = \Delta \ln(Y_{t+1}) $$
*Why? Because $\ln(Y_{t+1}) = \ln(Y_t(1+g)) = \ln(Y_t) + \ln(1+g)$, and for small $g$, $\ln(1+g) \approx g$.*

This approximation simplifies calculating **average annualized growth rates**:
$$ g_{\text{annualized}} = \left( \frac{Y_T}{Y_0} \right)^{1/T} - 1 \approx \frac{\ln(Y_T) - \ln(Y_0)}{T} $$
The log approximation is very accurate for small growth rates (e.g., typical annual GDP growth) and makes many derivations easier.

**Other Useful Log Rules:**
* $\ln(x \cdot y) = \ln(x) + \ln(y)$  (Growth rate of a product ≈ sum of growth rates)
* $\ln(x / y) = \ln(x) - \ln(y)$  (Growth rate of a ratio ≈ difference of growth rates)
* $\ln(x^a) = a \ln(x)$ (Used in deriving the Cobb-Douglas growth accounting equation)

The simulation below calculates the exact annualized growth rate and the log approximation. Notice how close they are for typical economic growth rates.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout
from IPython.display import display, Markdown
import warnings

def calculate_annualized_growth(Y0=100.0, YT=200.0, T=20):
    """
    Calculates the exact and log-approximated average annual growth rate
    between an initial value (Y0) and a final value (YT) over T years.

    Args:
        Y0 (float): Initial value (at time t=0).
        YT (float): Final value (at time t=T).
        T (int): Number of years between Y0 and YT.
    """
    # Input validation
    Y0 = max(Y0, 1e-9) # Avoid log(0) or division by zero
    YT = max(YT, 1e-9)
    T = int(T)
    if T < 1:
        display(Markdown("⚠️ Number of years (T) must be at least 1."))
        return

    # --- Calculations ---
    # Exact Annualized Growth Rate
    # g = (YT / Y0)**(1/T) - 1
    try:
        exact_g = (YT / Y0)**(1.0 / T) - 1.0
    except (ValueError, OverflowError):
        exact_g = np.nan # Handle potential errors if Y0/YT are extreme

    # Log Approximation Growth Rate
    # g_approx = (ln(YT) - ln(Y0)) / T
    try:
        log_approx_g = (np.log(YT) - np.log(Y0)) / T
    except (ValueError, OverflowError):
        log_approx_g = np.nan

    # Calculate difference
    if not (np.isnan(exact_g) or np.isnan(log_approx_g)):
        difference = abs(exact_g - log_approx_g)
        percent_diff = (difference / abs(exact_g)) * 100 if abs(exact_g) > 1e-9 else 0
    else:
        difference = np.nan
        percent_diff = np.nan

    # --- Display Results ---
    results_md = f"""
    ### 📈 Annualized Growth Rate Calculation:

    * **Initial Value (Y₀):** {Y0:,.2f}
    * **Final Value (Y<0xE1><0xB5><0x8_>):** {YT:,.2f}
    * **Number of Years (T):** {T}

    * **Exact Annualized Growth Rate (g):** {exact_g:.4%}
        * *Calculated as: $g = (Y_T / Y_0)^{{1/T}} - 1$*
    * **Log Approximation Growth Rate (g_approx):** {log_approx_g:.4%}
        * *Calculated as: $g \\approx [\\ln(Y_T) - \\ln(Y_0)] / T$*

    * **Difference (Exact - Approx):** {difference:.6f}
    * **Approximation Error:** {percent_diff:.2f}% of exact rate
        * *(Note: The log approximation is more accurate for smaller growth rates)*
    """
    display(Markdown(results_md))

    # --- Optional Plot: Value over Time ---
    # Show how the value grows at the calculated exact rate
    years_plot = np.arange(T + 1)
    Y_path = Y0 * (1 + exact_g)**years_plot

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.plot(years_plot, Y_path, marker='.', linestyle='-', color='teal')
    ax.set_xlabel("Years Elapsed")
    ax.set_ylabel("Value (Y)")
    ax.set_title(f"Growth Path at {exact_g:.2%} Average Annual Rate")
    ax.scatter([0, T], [Y0, YT], color='red', s=50, zorder=5, label='Start/End Points')
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
layout = Layout(width='95%')

interact(
    calculate_annualized_growth,
    Y0=FloatSlider(value=100.0, min=1, max=10000, step=10, description='Initial Value (Y0):', style=style, layout=layout, readout_format=',.0f'),
    YT=FloatSlider(value=200.0, min=1, max=20000, step=10, description='Final Value (YT):', style=style, layout=layout, readout_format=',.0f'),
    T=IntSlider(value=20, min=1, max=100, step=1, description='Number of Years (T):', style=style, layout=layout)
);


interactive(children=(FloatSlider(value=100.0, description='Initial Value (Y0):', layout=Layout(width='95%'), …

# 📘 Introduction to Growth Theory

Growth theory tries to explain two central things:
1. **Why are some countries richer than others?** (GDP per capita levels)
2. **Why do some countries grow faster than others?** (GDP per capita growth rates)

We distinguish:
- **Level** of GDP per capita: how rich a country is now
- **Growth rate** of GDP per capita: how fast it’s getting richer

💡 These two are **not the same** — and often **negatively correlated**:
- Poor countries can grow faster (convergence)
- Rich countries often have slow, steady growth

🔍 Example:
- The US has ~1.8% annual GDP/capita growth for 150+ years
- China grew at 5–10% for decades — but only recently became rich


# 📘 Preliminaries: Growth Math, Percent Changes, and Logs

We need tools to talk about growth:
- **Percent change** is the basis of the growth rate:
  \[ g = \frac{Y_{t+1} - Y_t}{Y_t} \]

- A **growth rate** is just a percent change written as a decimal:
  \[ g = 0.05 = 5\% \]

- **Annualized growth rates** give the average yearly rate over many years:
  \[ g_{Y,t\to s} = \left(\frac{Y_s}{Y_t}\right)^{1/(s-t)} - 1 \]

- Using **logs** makes this easier:
  \[ g \approx \frac{\ln Y_s - \ln Y_t}{s - t} \]


# 🧪 Example: GDP Growth Table
| Year | Level of GDP | Growth | Growth Rate |
|------|---------------|--------|--------------|
| 2018 | 100           | –      | –            |
| 2019 | 104           | +4     | 4.00%        |
| 2020 | 98            | –6     | –5.77%       |
| 2021 | 103           | +5     | 5.10%        |
| 2022 | 105           | +2     | 1.94%        |
| 2023 | 107           | +2     | 1.90%        |
| 2024 | 109           | +2     | 1.87%        |

Notice how the same growth in **level** results in **declining growth rates** as the base (denominator) increases.


# 📗 Quick Rules of Thumb
- \( \ln(1 + x) \approx x \) when \( x \) is small
- \( \ln(x^a) = a \ln x \)
- \( \frac{d \ln x}{dx} = \frac{1}{x} \)

We’ll use logs often in macro to simplify percent change math.


# Next: Measuring GDP — the components and flows of income/expenditure.
